### Importing libraries

In [69]:
import pandas as pd
import numpy as np
import datetime

### Importing data

In [128]:
ais=pd.read_csv(r'Product_1-vessel-movements-report-READY.csv')

In [129]:
unique_mmsi=ais['MMSI'].unique()

In [130]:
def preprocess(ais):
    '''
    Preprocessing date time column from AIS report
    '''
    ais['DATE TIME (UTC)']=pd.to_datetime(ais['DATE TIME (UTC)'])
    x=ais['DATE TIME (UTC)']-ais['DATE TIME (UTC)'].min()
    y=[]
    for i in range(len(x)):
        y.append(x[i].seconds)
    y=pd.DataFrame(y,columns=['seconds_diff'])

    ais_new=pd.concat([y,ais],axis=1)
    return ais_new

In [131]:
def speed_correction(ais_data, unique_mmsi):  
    '''
    Unreasonable stop during forward movement of vessels
    I.e., position of vessel remains constant even after having positive speed values. 
    If found, first data point out of two will be eliminated from data. Index of such data point will be provided as output.  
    '''
    index = []
    for i in unique_mmsi:
        l=ais_data.loc[ais_data.MMSI==i]
        l=l.reset_index()
        for j in range(len(l)-1):
            if ((l['LATITUDE'][j]==l['LATITUDE'][j+1])&(l['LONGITUDE'][j]==l['LONGITUDE'][j+1])&(l['SPEED']!=0)).all():
                index.append(l.iloc[j,0])
        
    return index

In [132]:
# Preprocessing and AIS data correction step 1

ais_new = preprocess(ais)
ais_new=ais_new.drop(speed_correction(ais_new, unique_mmsi),axis=0)

In [133]:

def distance_jump_correction(ais_data, unique_mmsi):
    '''
    Identifying unreasonable distance jump of any vessel from AIS data.
    To calculate distance between 2 data points, haversine formula is used. 
    Threshold of 15 meters/sec or 29.16 knots has been set for identifying anomalies. 
    Generally, most types of vessels considered in data travel at lower speed than 30 knots. This threshold also depends on geography.
    '''
    r=6378100
    m=[]
    for i in unique_mmsi:
        l=ais_new.loc[ais.MMSI==i]
        l=l.reset_index()
        if len(l)>1:
            for j in range(1,len(l)):
                a=np.radians(l['LATITUDE'][j])
                b=np.radians(l['LATITUDE'][j-1])
                c=np.radians(l['LONGITUDE'][j])
                d=np.radians(l['LONGITUDE'][j-1])
                dis=2*r*np.arcsin(np.sqrt((np.sin((a-b)/2))**2+np.cos(a)*np.cos(b)*(np.sin((c-d)/2))**2))
                sec=l['seconds_diff'][j]-l['seconds_diff'][j-1]
                if (dis/sec>15):
                    m.append(l['index'][j])
    
    return m
            
        

In [134]:
# AIS data correction step 2

ais_new=ais_new.drop(distance_jump_correction(ais_new, unique_mmsi),axis=0)

In [135]:
ais_new

,seconds_diff,DATE TIME (UTC),MMSI,LATITUDE,LONGITUDE,COURSE,SPEED,HEADING,NAVSTAT,IMO,...,AISTYPE,A,B,C,D,DRAUGHT,DESTINATION,ETA,LOA,TYPENAME
0,0,2019-11-27 11:14:00,533131038,1.30754,104.15111,273.0,0.1,3,1,9411288,...,84,87,23,7,12,5.5,PENGERANG,11-25 01:00,110,Chemical/Oil Products Tanker
1,0,2019-11-27 11:14:00,533170319,1.31791,104.08989,131.8,11.7,134,0,9101651,...,70,113,27,10,10,7.6,PORT_KLANG,11-28 11:30,140,Container Ship
2,0,2019-11-27 11:14:00,355902000,1.19623,105.01875,127.2,15.4,127,0,9202223,...,73,141,41,13,15,8.6,MY TPP>ID SUB,11-30 02:00,182,Container Ship
3,0,2019-11-27 11:14:00,477712400,1.41223,104.47880,49.9,18.2,50,0,9695121,...,71,159,241,36,24,12.6,QING DAO,12-02 18:00,400,Container Ship
4,1,2019-11-27 11:14:01,256728000,1.37139,104.63151,130.0,11.7,129,0,9670119,...,70,168,17,6,24,9.8,JAKARTA,11-30 00:01,185,Container Ship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,357,2019-11-27 11:19:57,533170354,1.33553,104.10318,298.7,0.2,60,0,9066186,...,80,70,20,2,11,5.5,NaN,02-26 05:00,90,Oil Products Tanker
1001,357,2019-11-27 11:19:57,538005592,1.31539,104.32986,229.1,7.8,235,0,9308417,...,79,205,17,15,15,9.0,SG PEBGA,11-27 14:00,222,Container Ship
1002,358,2019-11-27 11:19:58,405000249,1.25903,104.23605,79.0,12.2,74,0,9217682,...,70,160,30,20,11,7.0,TANJUNG KAMPEH,11-28 18:00,190,Bulk Carrier
1003,359,2019-11-27 11:19:59,356426000,1.12197,104.87265,312.3,15.2,314,0,9160401,...,79,0,0,0,0,10.5,SIN PEBGA,11-27 17:00,0,Container Ship


### AIS data interpolation

In [136]:
# SAR image acquisition time is 25 seconds. 
# The SAR image considered for integration is acquired on 27th Nov 2019 during 11:17:22 to 11:17:47 (period of 25 seconds)
# Midpoint of acquisition time is 11:17:35. 
# Difference between AIS data minimum date time and midpoint of SAR acquisition time is 215 seconds.
# midpoint of SAR acquisition time is also called as target interpolation time

sar_date_time = 215

In [137]:

def dead_reckoning_classification(ais_new, sar_date_time):
    '''
    AIS signal of each vessel is categorized into one of the two cases (according to its
    first and last signals present in AIS data) with respect to the target interpolation time (tSAR)
    
    1. AIS signal detected before and after target interpolation time
    2. AIS signal detected only before or only after target interpolation time
    '''
    
    type1 = pd.merge(ais_new.loc[ais_new.seconds_diff>sar_date_time]['MMSI'], ais_new.loc[ais_new.seconds_diff<sar_date_time]['MMSI'], how='inner', on=['MMSI'])['MMSI'].unique()
    type2 = np.array(list(set(ais_new.MMSI).difference(type1)))
    
    return type1, type2

In [138]:
# Linear Interpolation

def calculate_coordinates(lat1, lon1, bearing, speed, seconds, sar_date_time):
    '''
    Linear interpolation is used for type 2 ships. 
    '''
    R = 6371000
    d = speed*0.514444*(sar_date_time-seconds)
    

    lat1 = np.radians(lat1) #Current lat point converted to radians
    lon1 = np.radians(lon1) #Current long point converted to radians

    bearing = np.radians(bearing)

    lat2 = np.arcsin(np.sin(lat1)*np.cos(d/R) + np.cos(lat1)*np.sin(d/R)*np.cos(bearing))

    lon2 = lon1 + np.arctan2(np.sin(bearing)*np.sin(d/R)*np.cos(lat1), np.cos(d/R)-np.sin(lat1)*np.sin(lat2))

    lat2 = np.degrees(lat2)
    lon2 = np.degrees(lon2)

    return (round(lat2, 4), round(lon2, 4))


In [139]:
# Piecewise Hermite

def piecewise_hermite_lat(cor1,cor2,vel1,vel2,t1,t2,theta1,theta2, sar_date_time):
    '''
    Piecewise Hermite interpolation is used for type 1 ships. 
    '''
    a=np.array([[t1**3,t1**2,t1,1], [t2**3,t2**2,t2,1], [3*(t1**2),2*t1,1,0],[3*(t2**2),2*t1,1,0]])
    theta1=np.radians(theta1)
    theta2=np.radians(theta2)
    b=np.array([cor1,cor2,vel1*0.5014*np.sin(theta1)/111000,vel2*0.5014*np.sin(theta2)/111000])
    k=np.linalg.solve(a, b)
    lat=np.dot(k,[sar_date_time**3,sar_date_time**2,sar_date_time,1])
    return(lat)
    
    
def piecewise_hermite_lon(cor1,cor2,vel1,vel2,t1,t2,theta1,theta2, sar_date_time):
    '''
    Piecewise Hermite interpolation is used for type 1 ships. 
    '''
    a=np.array([[t1**3,t1**2,t1,1],[t2**3,t2**2,t2,1],[3*(t1**2),2*t1,1,0],[3*(t2**2),2*t1,1,0]])
    theta1=np.radians(theta1)
    theta2=np.radians(theta2)
    b=np.array([cor1,cor2,vel1*0.5014*np.cos(theta1)/111000,vel2*0.5014*np.cos(theta2)/111000])
    k=np.linalg.solve(a, b)
    lon=np.dot(k,[sar_date_time**3,sar_date_time**2,sar_date_time,1])
    return(lon)
    

In [140]:
new_data=pd.DataFrame()
type1, type2 = dead_reckoning_classification(ais_new, sar_date_time)

### Interpolating data for type 1 ships using piecewise hermite interpolation

In [141]:
for i in type1:
    
    d=ais_new[(ais_new['MMSI']==i)].reset_index(drop=True)
    index=(abs(d['seconds_diff']-sar_date_time)).idxmin()
    
    if d['seconds_diff'][index]>sar_date_time:
        i2=index
        i1=index-1
    else:
        i2=index+1
        i1=index
    
    lat1=piecewise_hermite_lat(d['LATITUDE'][i1],d['LATITUDE'][i2],d['SPEED'][i1],d['SPEED'][i2],d['seconds_diff'][i1],
                            d['seconds_diff'][i2],d['COURSE'][i1],d['COURSE'][i2], sar_date_time)
    lon1=piecewise_hermite_lon(d['LONGITUDE'][i1],d['LONGITUDE'][i2],d['SPEED'][i1],d['SPEED'][i2],d['seconds_diff'][i1],
                           d['seconds_diff'][i2],d['COURSE'][i1],d['COURSE'][i2], sar_date_time)
    le=d['A'][0]+d['B'][0]
    wi=d['C'][0]+d['D'][0]

    a=pd.DataFrame([[d['MMSI'][0],lat1,lon1,(d['seconds_diff'][i1]-314),d['SPEED'][i1],le,wi,d['NAME'][0],d['AISTYPE'][0]]],
                   columns=['MMSI','LATITUDE','LONGITUDE','Diff','SPEED_BEFORE','LENGTH','BREADTH','NAME','AISTYPE'])
                   
    new_data=pd.concat([new_data,a],axis=0,ignore_index=True)
    
    
              
              
    
    

### Interpolating data of type 2 ships using linear interpolation

In [142]:

for i in type2:
    l=ais_new[(ais_new['MMSI']==i)].reset_index(drop=True)
    min_index=(abs(l['seconds_diff']-sar_date_time)).idxmin()
    a=calculate_coordinates(l.loc[min_index]['LATITUDE'],l.loc[min_index]['LONGITUDE'],l.loc[min_index]['COURSE'],l.loc[min_index]['SPEED'], l.loc[min_index]['seconds_diff'], sar_date_time)
    b=pd.DataFrame([[l['MMSI'][0],a[0],a[1],(l['seconds_diff'][min_index]-sar_date_time),l['SPEED'][min_index],(l['A'][0]+l['B'][0]),(l['C'][0]+l['D'][0]),l['NAME'][0],l['AISTYPE'][0]]],columns=['MMSI','LATITUDE','LONGITUDE','Diff','SPEED_BEFORE','LENGTH','BREADTH','NAME','AISTYPE'])
    new_data=pd.concat([new_data,b],axis=0,ignore_index=True)

In [143]:
new_data

,MMSI,LATITUDE,LONGITUDE,Diff,SPEED_BEFORE,LENGTH,BREADTH,NAME,AISTYPE
0,373607000,1.368193,104.405203,-190,9.1,180,30,GLOBAL AMBITION,70
1,232011742,1.137939,104.091528,-106,7.1,327,57,BERGE LOGAN,70
2,533150037,1.308735,104.266332,-278,0.2,89,13,RUYI SATU,80
3,533170482,1.430440,104.481355,-219,14.8,154,25,DANUM 168,72
4,355902000,1.929797,105.031064,-104,15.4,182,28,UNI-PREMIER,73
...,...,...,...,...,...,...,...,...,...
66,371321000,1.337800,104.099500,-68,0.0,120,20,SUPER EASTERN,80
67,538004918,1.298900,104.093000,-47,0.2,361,65,SEA TUBARAO,70
68,563055800,1.535200,104.509900,-88,0.0,162,26,CHERRY,70
69,636091595,1.505700,104.824300,-82,0.1,294,32,CHICAGO,70


In [144]:
new_data.to_csv('AIS_interpolated_data.csv',index=False)